In [1]:
import sys
sys.path.insert(0,'ladybug')
sys.path.insert(0,'honeybee')

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [26]:
import ladybug
import honeybee

import json
import itertools
import re
import os

import eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
import numpy

In [28]:
# Load IDF ready for pre-processing and modification
idf_file = "test.idf"
epw_file = "weather/GBR_Cardiff_CIBSE_TRY.epw" 
idd_file = "C:/EnergyPlusV8-8-0/Energy+.idd"

IDF.setiddname(idd_file)
idf = IDF(idf_file)

In [54]:
#Load the EPW file to get the location variables
with open(epw_file, "r") as f:
    a = f.readlines()

a, name, c, d, e, f, g, h, i, j = a[0].replace("\n", "").split(",")

latitude = float(g)
longitude = float(h)

name

'CARDIFF'

In [33]:
# Parameters actually containing values 
[i for i in idf.idfobjects if len(idf.idfobjects[i]) != 0]

[u'SHADING:BUILDING:DETAILED',
 u'VERSION',
 u'OUTPUTCONTROL:TABLE:STYLE',
 u'SIZING:PARAMETERS',
 u'LIGHTS',
 u'WINDOWMATERIAL:GAS',
 u'MATERIAL',
 u'SIMULATIONCONTROL',
 u'MATERIAL:AIRGAP',
 u'DESIGNSPECIFICATION:OUTDOORAIR',
 u'PEOPLE',
 u'RUNPERIOD',
 u'SCHEDULE:WEEK:DAILY',
 u'ZONEINFILTRATION:DESIGNFLOWRATE',
 u'ZONE',
 u'ELECTRICEQUIPMENT',
 u'SIZINGPERIOD:DESIGNDAY',
 u'ZONEMIXING',
 u'OUTPUT:VARIABLEDICTIONARY',
 u'HVACTEMPLATE:THERMOSTAT',
 u'BUILDINGSURFACE:DETAILED',
 u'SITE:LOCATION',
 u'SCHEDULE:YEAR',
 u'FENESTRATIONSURFACE:DETAILED',
 u'TIMESTEP',
 u'GLOBALGEOMETRYRULES',
 u'SCHEDULE:DAY:INTERVAL',
 u'OUTPUT:SURFACES:LIST',
 u'HVACTEMPLATE:ZONE:IDEALLOADSAIRSYSTEM',
 u'CONSTRUCTION',
 u'OUTPUT:VARIABLE',
 u'SHADOWCALCULATION',
 u'WINDOWMATERIAL:GLAZING',
 u'BUILDING',
 u'SCHEDULETYPELIMITS']

In [35]:
# 
idf.idfobjects["SITE:LOCATION"]

[
Site:Location,
    LONDON/GATWICK	GBR,       !- Name
    51.15,                    !- Latitude
    -0.18,                    !- Longitude
    0.0,                      !- Time Zone
    62.0;                     !- Elevation
]

# Below this = TESTING/NOTES

In [20]:
# Load IDF into a list of lists (using basic text)

idf_filepath = "test.idf"

def isplit(iterable, splitters):
    return [list(g) for k,g in itertools.groupby(iterable,lambda x:x in splitters) if not k]

def nestrepl(lst, what, repl):
    for index, item in enumerate(lst):
        if type(item) == list:
            nestrepl(item, what, repl)
        else:
            if item == what:
                lst[index] = repl

with open(idf_filepath, "r") as f:
    content = isplit(isplit([re.sub("[ ]{2,}", "", re.sub("\r\n", "", re.sub("\t", "", x))) for x in f.readlines()], "")[0], "\n")
    content = [[x.replace('\n','') for x in l] for l in content]


* If length of sublist is 1 then only one value e.g. Timestep
* If sublist length of >1 then multiple values
* If count of sublist unique first values (if > 1) 


In [ ]:
Modify the following entries:


In [25]:
# Remove unecessary parts
a = []
for i in content:
    if i[0] not in ["SizingPeriod:DesignDay,"]:
        a.append(i)
    else:
        pass
a

[['Version, ;'],
 ['Timestep, 6;'],
 ['ShadowCalculation,',
  'AverageOverDaysInFrequency,!- Calculation Method',
  '30,!- Calculation Frequency',
  '3000;!- Maximum Figures in Shadow Overlap Calculation'],
 ['Building,',
  'test.idf, !- Name',
  '0.0, !- North Axis {deg}',
  'City, !- Terrain',
  ', !- Loads Convergence Tolerance Value',
  ', !- Temperature Convergence Tolerance Value {deltaC}',
  'FullInteriorAndExteriorWithReflections, !- Solar Distribution or maybe FullExterior',
  ', !- Maximum Number of Warmup Days',
  '; !- Minimum Number of Warmup Days'],
 ['Sizing:Parameters,',
  '1.25,!- Heating Sizing Factor',
  '1.15;!- Cooling Sizing Factor'],
 ['Site:Location,',
  'LONDON/GATWICKGBR,',
  '51.15,!Latitude',
  '-0.18,!Longitude',
  '0.0, !Time Zone',
  '62.0;!Elevation'],
 ['SimulationControl,',
  'Yes,!- Do Zone Sizing Calculation',
  'Yes,!- Do System Sizing Calculation',
  'Yes,!- Do Plant Sizing Calculation',
  'No,!- Run Simulation for Sizing Periods',
  'Yes; !- Run S

In [22]:
[i[0] for i in content]

['Version, ;',
 'Timestep, 6;',
 'ShadowCalculation,',
 'Building,',
 'Sizing:Parameters,',
 'Site:Location,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SimulationControl,',
 'RunPeriod,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'Sched